In [ ]:
import collections
from collections import Counter, defaultdict, OrderedDict, namedtuple, deque, ChainMap
from collections import UserDict, UserList, UserString
import time
import json
# =====================================================
# SECTION 9: REAL-WORLD PROJECT EXAMPLES
# =====================================================

print("\n\n9. REAL-WORLD PROJECT EXAMPLES")
print("-" * 50)

# Example 1: Web Server Request Analysis
print("🌐 Example 1: Web Server Log Analysis")

class WebLogAnalyzer:
    def __init__(self):
        self.ip_counter = Counter()
        self.status_counter = Counter()
        self.url_counter = Counter()
        self.hourly_requests = defaultdict(int)
        self.recent_requests = deque(maxlen=1000)  # Keep last 1000 requests
    
    def process_log_entry(self, log_entry):
        """Process a single log entry"""
        # Simulate parsing: IP - - [timestamp] "GET /url HTTP/1.1" status size
        parts = log_entry.split()
        if len(parts) >= 7:
            ip = parts[0]
            timestamp = parts[3][1:]  # Remove [
            method_url = parts[5] + " " + parts[6]
            status = parts[8]
            
            # Update counters
            self.ip_counter[ip] += 1
            self.status_counter[status] += 1
            self.url_counter[method_url] += 1
            
            # Extract hour from timestamp
            hour = timestamp.split(':')[1] if ':' in timestamp else '00'
            self.hourly_requests[hour] += 1
            
            # Add to recent requests
            self.recent_requests.append({
                'ip': ip,
                'timestamp': timestamp,
                'url': method_url,
                'status': status
            })
    
    def get_report(self):
        """Generate analysis report"""
        return {
            'top_ips': self.ip_counter.most_common(5),
            'status_distribution': dict(self.status_counter),
            'popular_urls': self.url_counter.most_common(3),
            'peak_hours': sorted(self.hourly_requests.items(), 
                               key=lambda x: x[1], reverse=True)[:3],
            'total_requests': sum(self.ip_counter.values())
        }

# Test web log analyzer
log_entries = [
    '192.168.1.1 - - [01/Jan/2024:10:00:00 +0000] "GET /index.html HTTP/1.1" 200 1234',
    '192.168.1.2 - - [01/Jan/2024:10:15:00 +0000] "GET /about.html HTTP/1.1" 200 567',
    '192.168.1.1 - - [01/Jan/2024:11:30:00 +0000] "POST /login HTTP/1.1" 302 0',
    '192.168.1.3 - - [01/Jan/2024:11:45:00 +0000] "GET /index.html HTTP/1.1" 200 1234',
    '192.168.1.2 - - [01/Jan/2024:12:00:00 +0000] "GET /nonexistent HTTP/1.1" 404 0',
    '192.168.1.1 - - [01/Jan/2024:12:30:00 +0000] "GET /index.html HTTP/1.1" 200 1234'
]

analyzer = WebLogAnalyzer()
for entry in log_entries:
    analyzer.process_log_entry(entry)

report = analyzer.get_report()
print(f"Total requests: {report['total_requests']}")
print(f"Top IPs: {report['top_ips']}")
print(f"Status codes: {report['status_distribution']}")
print(f"Popular URLs: {report['popular_urls']}")

# Example 2: Game Score Tracking System
print("\n🎮 Example 2: Game Score Tracking System")

Player = namedtuple('Player', 'id name level score')

class GameScoreTracker:
    def __init__(self, max_leaderboard=10):
        self.players = {}  # player_id -> Player
        self.score_history = defaultdict(lambda: deque(maxlen=50))  # Recent scores
        self.leaderboard = deque(maxlen=max_leaderboard)
        self.level_distribution = Counter()
        self.achievements = defaultdict(set)
    
    def add_player(self, player_id, name):
        """Add new player"""
        player = Player(player_id, name, 1, 0)
        self.players[player_id] = player
        self.level_distribution[1] += 1
    
    def update_score(self, player_id, new_score):
        """Update player score"""
        if player_id not in self.players:
            return False
        
        old_player = self.players[player_id]
        self.score_history[player_id].append(old_player.score)
        
        # Create new player record with updated score
        new_player = old_player._replace(score=new_score)
        self.players[player_id] = new_player
        
        # Update leaderboard
        self._update_leaderboard(new_player)
        
        # Check for achievements
        self._check_achievements(player_id, old_player.score, new_score)
        
        return True
    
    def level_up(self, player_id):
        """Level up a player"""
        if player_id not in self.players:
            return False
        
        old_player = self.players[player_id]
        self.level_distribution[old_player.level] -= 1
        
        new_player = old_player._replace(level=old_player.level + 1)
        self.players[player_id] = new_player
        self.level_distribution[new_player.level] += 1
        
        return True
    
    def _update_leaderboard(self, player):
        """Update the leaderboard with new score"""
        # Remove player if already in leaderboard
        self.leaderboard = deque([p for p in self.leaderboard if p.id != player.id], 
                                maxlen=self.leaderboard.maxlen)
        
        # Insert player in correct position
        inserted = False
        new_leaderboard = deque(maxlen=self.leaderboard.maxlen)
        
        for existing_player in self.leaderboard:
            if not inserted and player.score > existing_player.score:
                new_leaderboard.append(player)
                inserted = True
            new_leaderboard.append(existing_player)
        
        if not inserted and len(new_leaderboard) < new_leaderboard.maxlen:
            new_leaderboard.append(player)
        
        self.leaderboard = new_leaderboard
    
    def _check_achievements(self, player_id, old_score, new_score):
        """Check for score-based achievements"""
        milestones = [100, 500, 1000, 5000, 10000]
        
        for milestone in milestones:
            if old_score < milestone <= new_score:
                self.achievements[player_id].add(f"Score {milestone}")
    
    def get_stats(self):
        """Get game statistics"""
        return {
            'total_players': len(self.players),
            'level_distribution': dict(self.level_distribution),
            'leaderboard': [(p.name, p.score) for p in self.leaderboard],
            'avg_score': sum(p.score for p in self.players.values()) / len(self.players) if self.players else 0
        }

# Test game score tracker
tracker = GameScoreTracker(max_leaderboard=5)

# Add players
tracker.add_player(1, "Alice")
tracker.add_player(2, "Bob")
tracker.add_player(3, "Charlie")
tracker.add_player(4, "Diana")

# Update scores
score_updates = [
    (1, 150), (2, 200), (3, 75), (4, 300),
    (1, 250), (2, 180), (3, 400), (4, 520),
    (1, 600), (2, 750)
]

for player_id, score in score_updates:
    tracker.update_score(player_id, score)

# Level up some players
tracker.level_up(1)
tracker.level_up(2)

stats = tracker.get_stats()
print(f"Game Statistics:")
print(f"  Total players: {stats['total_players']}")
print(f"  Average score: {stats['avg_score']:.1f}")
print(f"  Level distribution: {stats['level_distribution']}")
print(f"  Leaderboard: {stats['leaderboard']}")

# Show achievements
print(f"  Achievements earned:")
for player_id, achievements in tracker.achievements.items():
    player_name = tracker.players[player_id].name
    print(f"    {player_name}: {list(achievements)}")




4. NAMEDTUPLE - TUPLE WITH NAMED FIELDS
--------------------------------------------------
🏷️ namedtuple creates tuple subclasses with named fields
   Provides readable, lightweight object-like access

✅ Basic namedtuple Examples:
Point: Point(x=10, y=20)
X coordinate: 10
Y coordinate: 20
Person: Person(name='Alice', age=30, city='New York')
Name: Alice, Age: 30

🔧 namedtuple Methods:
As dictionary: {'name': 'Alice', 'age': 30, 'city': 'New York'}
Original: Person(name='Alice', age=30, city='New York')
Modified: Person(name='Alice', age=31, city='Boston')
Point fields: ('x', 'y')
Person fields: ('name', 'age', 'city')
Point from list: Point(x=30, y=40)
Student with defaults: Student(name='Bob', grade='A', age=18), Student(name='Charlie', grade='B', age=18), Student(name='David', grade='A', age=19)

🌍 Real-world Examples:
Employee Records:
  Alice Johnson (Engineering): $75000
  Bob Smith (Marketing): $65000
  Charlie Brown (Engineering): $80000

Average salary by department:
  Engine